In [1]:
from __future__ import division, print_function, absolute_import

import tensorflow as tf

In [2]:
""" Convolutional Neural Network.
Build and train a convolutional neural network with TensorFlow.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)
Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
"""

# Training Parameters
num_epochs = 100
batch_size = 64
num_classes = 6 # MNIST total classes (0-9 digits)


img_size = 224

In [3]:
import os
import numpy as np
from IPython import display
from PIL import Image
from glob import glob

filenames = []
labels = []

train_dir = '/home/ifranco/tf_real_estate_images_classification/data/processed/vision_based_dataset/train/'

for d in os.listdir(train_dir):
    category_filenames = glob(f'{train_dir}/{d}/*')
    filenames.extend(category_filenames)
    labels.extend([d for x in range(len(category_filenames))])

mapping = {l: idx for idx, l in enumerate(np.unique(labels))}

category = [mapping[l] for l in labels]

In [4]:
category = tf.keras.utils.to_categorical(category, num_classes)

In [5]:

# step 1
filenames = tf.constant(filenames)
labels = tf.constant(category)

# step 2: create a dataset returning slices of `filenames`
dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))

In [6]:

# step 3: parse every image in the dataset using `map`
def _parse_function(filename, label):
    image_string = tf.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image_decoded, tf.float32)
    # resized_images = tf.image.resize_images(images, (img_size, img_size))
    resized_image = tf.image.resize_images(image, (img_size, img_size))
    # resized_image = tf.reshape(
    #     resized_image,
    #     (img_size * img_size * 3, ))
    return resized_image, label

dataset = dataset.map(_parse_function)
dataset = dataset.shuffle(5000, reshuffle_each_iteration=True).repeat(100).batch(batch_size)

In [7]:
# step 4: create iterator and final input tensor
iterator = tf.compat.v1.data.make_one_shot_iterator(dataset)
images, labels = iterator.get_next()

In [8]:
learning_rate = 0.001
# num_steps = batch_size * num_epochs
num_steps = int(100 * 4500 / batch_size) - 1
display_step = 50

# Network Parameters
num_input = (img_size * img_size * 3) # MNIST data input (img shape: 28*28)
# num_input = (img_size, img_size, 3) # * 128 # MNIST data input (img shape: 28*28)
dropout = 0.5 # Dropout, probability to keep units

# tf Graph input
X = tf.placeholder(tf.float32, [None, img_size, img_size, 3], name='X')
Y = tf.placeholder(tf.float32, [None, num_classes], name='y')
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)


In [9]:

# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

# Create model
def conv_net(x, weights, biases, dropout):
    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    # x = tf.reshape(x, shape=[-1, 224, 224, 3]) 
    
    # Convolution Layer
    conv1 = conv2d(x, weights['block_1_wc1'], biases['block_1_bc1'])
    conv1 = tf.compat.v1.layers.batch_normalization(conv1, 
                                                    training=True if dropout != 1 else False)
    conv1 = conv2d(conv1, weights['block_1_wc2'], biases['block_1_bc2'])
    conv1 = tf.compat.v1.layers.batch_normalization(conv1,
                                                    training=True if dropout != 1 else False)
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)
    # Apply Dropout
    conv1 = tf.nn.dropout(conv1, 1 if dropout == 1 else 0.75)
    
    # Convolution Layer
    conv2 = conv2d(conv1, weights['block_2_wc1'], biases['block_2_bc1'])
    conv2 = tf.compat.v1.layers.batch_normalization(conv2, 
                                                    training=True if dropout != 1 else False)
    conv2 = conv2d(conv2, weights['block_2_wc2'], biases['block_2_bc2'])
    conv2 = tf.compat.v1.layers.batch_normalization(conv2,
                                                    training=True if dropout != 1 else False)
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)
    # Apply Dropout
    conv2 = tf.nn.dropout(conv2, 1 if dropout == 1 else 0.75)
        
    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'block_1_wc1': tf.Variable(tf.random_normal([3, 3, 3, 32])),
    'block_1_wc2': tf.Variable(tf.random_normal([3, 3, 32, 32])),
    
    'block_2_wc1': tf.Variable(tf.random_normal([3, 3, 32, 64])),
    'block_2_wc2': tf.Variable(tf.random_normal([3, 3, 64, 64])),
    
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*4096, 512])),
    # 1024 inputs, 6 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([512, num_classes]))
}

biases = {
    'block_1_bc1': tf.Variable(tf.random_normal([32])),
    'block_1_bc2': tf.Variable(tf.random_normal([32])),
    'block_2_bc1': tf.Variable(tf.random_normal([64])),
    'block_2_bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([512])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [10]:
# Construct model
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

W1130 19:46:43.159493 139724735682368 deprecation.py:323] From <ipython-input-9-849358fdedb9>:24: batch_normalization (from tensorflow.python.layers.normalization) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
W1130 19:46:43.269107 139724735682368 deprecation.py:506] From <ipython-input-9-849358fdedb9>:31: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1130 19:46:43.350522 139724735682368 deprecation.py:323] From <ipython-input-10-b9007dae97ad>:7: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version

In [11]:
from datetime import datetime as dt

In [12]:
import numpy as np

In [13]:
import tensorflow as tf

In [14]:
saver = tf.train.Saver(max_to_keep=4, keep_checkpoint_every_n_hours=2)

In [15]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    start = dt.now()
    
    for step in range(1, num_steps+1):
        #batch_x, batch_y = mnist.train.next_batch(batch_size)
        # batch_x, batch_y = iterator.get_next()
        batch_x, batch_y = sess.run([images, labels])
        batch_start = dt.now()
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: 0.5})
        
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy],
                             feed_dict={
                                 X: batch_x,
                                 Y: batch_y,
                                 keep_prob: 1.0}
                            )
            end_display = (dt.now() - batch_start).total_seconds() / 60
            print(f"Step {step}/{num_steps} ({step*batch_size} imgs) >> minibatch loss: {np.round(loss, 2)}; accuracy {np.round(acc, 2)} (Total mins: {np.round((dt.now() - start).total_seconds() / 60, 2)}. ETA from prev: {end_display}.)")

    saved_path = saver.save(sess, './my-model', global_step=step)

    print("Optimization Finished!")

Step 1/7030 (64 imgs) >> minibatch loss: 1665.510009765625; accuracy 0.800000011920929 (Total mins: 0.24. ETA from prev: 0.049974866666666666.)
Step 50/7030 (3200 imgs) >> minibatch loss: 0.0; accuracy 1.0 (Total mins: 0.57. ETA from prev: 0.005970016666666666.)
Step 100/7030 (6400 imgs) >> minibatch loss: 0.0; accuracy 1.0 (Total mins: 0.9. ETA from prev: 0.005969083333333333.)
Step 150/7030 (9600 imgs) >> minibatch loss: 0.0; accuracy 1.0 (Total mins: 1.23. ETA from prev: 0.0059567.)
Step 200/7030 (12800 imgs) >> minibatch loss: 7225.14013671875; accuracy 0.949999988079071 (Total mins: 1.56. ETA from prev: 0.005980716666666667.)
Step 250/7030 (16000 imgs) >> minibatch loss: 2237.929931640625; accuracy 0.5899999737739563 (Total mins: 1.96. ETA from prev: 0.005983550000000001.)
Step 300/7030 (19200 imgs) >> minibatch loss: 763.8599853515625; accuracy 0.7200000286102295 (Total mins: 2.36. ETA from prev: 0.006063799999999999.)
Step 350/7030 (22400 imgs) >> minibatch loss: 545.51000976562

Step 2900/7030 (185600 imgs) >> minibatch loss: 1.7799999713897705; accuracy 0.05999999865889549 (Total mins: 22.3. ETA from prev: 0.005975016666666667.)
Step 2950/7030 (188800 imgs) >> minibatch loss: 1.850000023841858; accuracy 0.0 (Total mins: 22.62. ETA from prev: 0.005999183333333334.)
Step 3000/7030 (192000 imgs) >> minibatch loss: 3.4200000762939453; accuracy 0.0 (Total mins: 22.95. ETA from prev: 0.0059698166666666665.)
Step 3050/7030 (195200 imgs) >> minibatch loss: 2.5799999237060547; accuracy 0.019999999552965164 (Total mins: 23.28. ETA from prev: 0.005987500000000001.)
Step 3100/7030 (198400 imgs) >> minibatch loss: 1.4199999570846558; accuracy 0.0 (Total mins: 23.6. ETA from prev: 0.005984183333333334.)
Step 3150/7030 (201600 imgs) >> minibatch loss: 1.3300000429153442; accuracy 0.9800000190734863 (Total mins: 23.93. ETA from prev: 0.0060002499999999995.)
Step 3200/7030 (204800 imgs) >> minibatch loss: 2.0199999809265137; accuracy 0.4399999976158142 (Total mins: 24.31. ETA

Step 5700/7030 (364800 imgs) >> minibatch loss: 1.8300000429153442; accuracy 0.0 (Total mins: 42.76. ETA from prev: 0.005989316666666666.)
Step 5750/7030 (368000 imgs) >> minibatch loss: 1.75; accuracy 0.0 (Total mins: 43.15. ETA from prev: 0.005982749999999999.)
Step 5800/7030 (371200 imgs) >> minibatch loss: 1.649999976158142; accuracy 0.05000000074505806 (Total mins: 43.54. ETA from prev: 0.005999866666666667.)
Step 5850/7030 (374400 imgs) >> minibatch loss: 1.5499999523162842; accuracy 0.9800000190734863 (Total mins: 43.93. ETA from prev: 0.00598775.)
Step 5900/7030 (377600 imgs) >> minibatch loss: 1.4800000190734863; accuracy 0.949999988079071 (Total mins: 44.32. ETA from prev: 0.005985183333333334.)
Step 5950/7030 (380800 imgs) >> minibatch loss: 1.6399999856948853; accuracy 0.5600000023841858 (Total mins: 44.7. ETA from prev: 0.005970883333333333.)
Step 6000/7030 (384000 imgs) >> minibatch loss: 1.7200000286102295; accuracy 0.3100000023841858 (Total mins: 45.09. ETA from prev: 0

In [ ]:

# Calculate accuracy for 256 MNIST test images
print("Testing Accuracy:", \
    sess.run(accuracy, feed_dict={X: mnist.test.images[:256],
                                  Y: mnist.test.labels[:256],
                                  keep_prob: 1.0}))